Notebook with functions for reading in data and getting ensemble means for various experiments (0.1, 0.2, and 0.5 TW; surf, therm, mid, and bot; const, 2xCO2, and 4xCO2). This is designed to be used with the notebook read_and_calculate.ipynb.

In [1]:
import numpy as np
import xarray as xr

# modules for using datetime variables
import datetime
from datetime import time

import warnings
warnings.filterwarnings('ignore')

import cftime
from pandas.errors import OutOfBoundsDatetime  # Import the specific error

from xclim import ensembles

In [2]:
myVars = globals()

# Functions

In [3]:
# def get_const_CO2_ens_dat(avg_period,mem1_start,mem1_end,var_list,verbose=False):

#     """
#     Returns variables containing the ensemble-mean raw data and variables containing the ensemble-mean anomaly data. Anomalies are
#     calculated as the difference relative to the control run during the period corresponding to an ensemble member (i.e. the anomalies
#     for ensemble member 2 for year 201 to 400 are taking as the difference relative to year 201 to 400 of the control run).

#         Args:
#             avg_period (int): number of years for av/ts period
#             mem1_start (int)
#             mem1_end (int)
#             var_list (str list): list of variables to read (e.g. var_list = ["temp", "N2", "age", "rhopot2", "salt"])
#         Returns:
#             has no return variables, but creates xarray datasets by using myVars = globals()
            
#     """
    # const_exp_root = '_1860IC_200yr_'

#     mem2_start = mem1_start + 200
#     mem2_end = mem1_end + 200

#     mem3_start = mem1_start + 400
#     mem3_end = mem1_end + 400

#     # control
#     ctrl_mem1 = get_pp_av_data('tune_ctrl_1860IC_200yr',mem1_start,mem1_end,avg_period,pp_type='av-annual')
#     ctrl_mem2 = get_pp_av_data('tune_ctrl_1860IC_201-2001',mem2_start,mem2_end,avg_period,pp_type='av-annual',time_decoding=False)
#     ctrl_mem3 = get_pp_av_data('tune_ctrl_1860IC_201-2001',mem3_start,mem3_end,avg_period,pp_type='av-annual',time_decoding=False)

#     for idx, time_val in enumerate(ctrl_mem1.time.values):
#         ctrl_mem2.time.values[idx] = time_val
#         ctrl_mem3.time.values[idx] = time_val

#     raw_ctrl = ensembles.create_ensemble([ctrl_mem1,ctrl_mem2,ctrl_mem3]).mean("realization")
#     raw_ctrl = raw_ctrl[var_list]
#     raw_ctrl_name = f"const_ctrl_{mem1_start}_{mem1_end}"
#     myVars.__setitem__(raw_ctrl_name, raw_ctrl)

#     if verbose:
#         print(f"{raw_ctrl_name} done")
    
#     for prof in profiles:
#         for idx, power_str in enumerate(power_inputs):

#             mem1_exp_name = prof+const_exp_root+power_str
#             mem1_ds = get_pp_av_data(mem1_exp_name,mem1_start,mem1_end,avg_period,pp_type='av-annual')

#             if mem1_start == 1 and mem1_end == 25:
#                 if prof == "therm" and power_str == "0.5TW":
#                     continue #mem3 pp missing
#                 elif prof == "mid":
#                     continue #mem3 pp missing
#                 elif prof == "bot":
#                     continue

#             mem2_exp_name = "mem2_"+prof+const_exp_root+power_str
#             mem2_ds = get_pp_av_data(mem2_exp_name,mem2_start,mem2_end,avg_period,pp_type='av-annual',time_decoding=False)

#             mem3_exp_name = "mem3_"+prof+const_exp_root+power_str
#             mem3_ds = get_pp_av_data(mem3_exp_name,mem3_start,mem3_end,avg_period,pp_type='av-annual',time_decoding=False)

#             for i, time_val in enumerate(mem1_ds.time.values):
#                 mem2_ds.time.values[i] = time_val
#                 mem3_ds.time.values[i] = time_val
                
#             raw_ens = ensembles.create_ensemble([mem1_ds,mem2_ds,mem3_ds]).mean("realization")
#             raw_ens = raw_ens[var_list]
#             raw_ens_name = f"{prof}_{power_var_suff[idx]}_{mem1_start}_{mem1_end}"
#             myVars.__setitem__(raw_ens_name, raw_ens)

#             mem1_diff = mem1_ds[var_list] - ctrl_mem1[var_list]
#             mem2_diff = mem2_ds[var_list] - ctrl_mem2[var_list]
#             mem3_diff = mem3_ds[var_list] - ctrl_mem3[var_list]

#             diff_ens = ensembles.create_ensemble([mem1_diff,mem2_diff,mem3_diff])
#             diff_ens = diff_ens[var_list]
#             diff_ens['wet'] = ctrl_mem1['wet']

#             for var in var_list:
#                 cond1 = (diff_ens[var] > 0).all(dim="realization")
#                 cond2 = (diff_ens[var] < 0).all(dim="realization")
#                 nan_cond = np.isnan(diff_ens[var])

#                 # Identify regions where members agree or where it's NaN (i.e., land)
#                 agreement_maskbin = cond1 | cond2 | nan_cond

#                 # Set hatching mask: True where they disagree, False where they agree or it's NaN (over land or bathymetry)
#                 agreement_mask = xr.where(agreement_maskbin, False, True)
                
#                 # Store in dataset
#                 mask_name = f"{var}_agreement"
#                 diff_ens[mask_name] = agreement_mask

#             diff_ens = diff_ens.mean("realization")
#             diff_ens['wet'] = ctrl_mem1['wet']
#             diff_ens['dxt'] = ctrl_mem1['dxt']
#             diff_ens['dyt'] = ctrl_mem1['dyt']
#             diff_ens['areacello'] = ctrl_mem1['areacello']

#             # alternative to get more stats is:
#             # diff_ens = ensembles.ensemble_mean_std_max_min(diff_ens)
#             # for var in var_list:
#             #     mean_var_name = var + '_mean'
#             #     diff_ens = diff_ens.rename({mean_var_name: var})

#             diff_ens_name = f"{raw_ens_name}_diff"
#             myVars.__setitem__(diff_ens_name, diff_ens)
    
#             horiz_avg_diff_name = f"{diff_ens_name}_mean"
#             horiz_avg_diff = xr.Dataset()
#             for var in var_list:
#                 horiz_avg_diff[var] = horizontal_mean(myVars[diff_ens_name][var],mem1_ds)
#             myVars.__setitem__(horiz_avg_diff_name, horiz_avg_diff)

#             if verbose:
#                 print(f'{raw_ens_name}, {diff_ens_name}, {horiz_avg_diff_name} done')

## Functions to return ensemble and ensemble mean data

In [4]:
def create_const_doub_ens_mean(exp_name_list,start_years,end_years,chunk_length,variable_list,verbose=False):

    num_ens_mem = 3
    ens_mem_list = [None] * num_ens_mem
    
    for idx, exp_name in enumerate(exp_name_list):
        if idx == 0:
            time_decoding=True
        else:
            time_decoding=False
            
        ens_mem_list[idx] = get_pp_av_data(exp_name,start_years[idx],end_years[idx],chunk_length,pp_type='av-annual',\
                                                  time_decoding=time_decoding)

    # adjust timestamps to range of year 1 to 200
    for idx, time_val in enumerate(ens_mem_list[0].time.values):
        ens_mem_list[1].time.values[idx] = time_val
        ens_mem_list[2].time.values[idx] = time_val

    # for i in range(num_ens_mem):
    #     ens_mem_list[i] = ens_mem_list[i][variable_list]
        
    # ensemble mean
    ens_mean = ensembles.create_ensemble(ens_mem_list).mean("realization")
    ens_mean = ens_mean[variable_list]

    return ens_mem_list, ens_mean

In [5]:
def create_quad_ens_mean(exp_name_list,doub_ens_mem_list,doub_ens_mean,start_years,end_years,chunk_length,
                              variable_list,verbose=False):
    num_ens_mem = 3
    quad_ens_mem_list = [None] * num_ens_mem
    
    doub_cutoff_yr = 51
    doub_cutoff_dt = cftime.DatetimeNoLeap(doub_cutoff_yr, 1, 1, 0, 0, 0, 0, has_year_zero=True)
    post_51_start_years = [51,251,451]
    
    if start_years[0] < doub_cutoff_yr and end_years[0] < doub_cutoff_yr:
        quad_ens_mem_list = doub_ens_mem_list
        quad_ens_mean = doub_ens_mean
        
    elif start_years[0] < doub_cutoff_yr and end_years[0] > doub_cutoff_yr:
        for idx, ens_memb in enumerate(doub_ens_mem_list):
            if idx == 0:
                time_decoding=True
            else:
                time_decoding=False

            # all the 2xCO2 ensemble members have already had their times adjusted to within 1-200 range
            pre_51_quad = ens_memb.sel(time=slice(None,doub_cutoff_dt))
            
            if len(pre_51_quad) == 0:
                raise ValueError("len(pre_51_quad) = 0. It seems that 2xCO2 members have not had their times \
                adjusted to be within 1-200 years.")
            
            if verbose:
                print(f"pre_51_quad.time.values: {pre_51_quad.time.values}")
        
            post_51_quad = get_pp_av_data(exp_name_list[idx],post_51_start_years[idx],end_years[idx],chunk_length,pp_type='av-annual',\
                                         time_decoding=time_decoding)
            post_51_quad = post_51_quad[variable_list]
            quad_ens_mem_list[idx] = xr.concat([pre_51_quad,post_51_quad],"time")
    
    else: # a.k.a. if (start_years[0] > doub_cutoff_yr)
        for idx, ens_memb in enumerate(doub_ens_mem_list):
            if idx == 0:
                time_decoding=True
            else:
                time_decoding=False
                
            quad_ens_mem_list[idx] = get_pp_av_data(exp_name_list[idx],start_years[idx],end_years[idx],chunk_length,pp_type='av-annual',\
                                         time_decoding=time_decoding)

    # adjust timestamps to range of year 1 to 200
    for idx, time_val in enumerate(quad_ens_mem_list[0].time.values):
        quad_ens_mem_list[1].time.values[idx] = time_val
        quad_ens_mem_list[2].time.values[idx] = time_val

    # for i in range(num_ens_mem):
    #     quad_ens_mem_list[i] = quad_ens_mem_list[i][variable_list]
                
    # ensemble mean
    quad_ens_mean = ensembles.create_ensemble(quad_ens_mem_list).mean("realization")
    quad_ens_mean = quad_ens_mean[variable_list]
    
    return quad_ens_mem_list, quad_ens_mean

## Function to calculate ensemble-mean differences and horizontal mean differences

In [6]:
def calc_ens_diffs(diff_ens_name,ref_ens_list,perturb_ens_list,variable_list,verbose=False):

    num_ens_mem = 3

    diffs_mem_list = [None] * num_ens_mem
    
    for i in range(num_ens_mem):
        diffs_mem_list[i] = perturb_ens_list[i][variable_list] - ref_ens_list[i][variable_list]

    diff_ens = ensembles.create_ensemble(diffs_mem_list)
    diff_ens['wet'] = ref_ens_list[0]['wet']

    for var in variable_list:
        cond1 = (diff_ens[var] > 0).all(dim="realization")
        cond2 = (diff_ens[var] < 0).all(dim="realization")
        nan_cond = np.isnan(diff_ens[var])

        # Identify regions where members agree or where it's NaN (i.e., land)
        hatching_maskbin = cond1 | cond2 | nan_cond

        # Set hatching mask: True where they disagree, False where they agree or it's NaN (over land or bathymetry)
        hatching_mask = xr.where(hatching_maskbin, False, True)
        
        # Store in dataset
        mask_name = f"{var}_disagree"
        diff_ens[mask_name] = hatching_mask

    diff_ens = diff_ens.mean("realization")
    diff_ens['wet'] = ref_ens_list[0]['wet']
    diff_ens['dxt'] = ref_ens_list[0]['dxt']
    diff_ens['dyt'] = ref_ens_list[0]['dyt']
    diff_ens['areacello'] = ref_ens_list[0]['areacello']
    myVars.__setitem__(diff_ens_name, diff_ens)

    # alternative to get more stats is:
    # diff_ens = ensembles.ensemble_mean_std_max_min(diff_ens)
    # for var in variable_list:
    #     mean_var_name = var + '_mean'
    #     diff_ens = diff_ens.rename({mean_var_name: var})

    horiz_avg_diff_name = f"{diff_ens_name}_mean"
    horiz_avg_diff = xr.Dataset()
    for var in variable_list:
        horiz_avg_diff[var] = horizontal_mean(diff_ens[var],diff_ens)
    myVars.__setitem__(horiz_avg_diff_name, horiz_avg_diff)

    if verbose:
        print(f'{diff_ens_name}, {horiz_avg_diff_name} done')

## Main function

In [7]:
def get_ens_dat(co2_scen, avg_period, mem1_start, mem1_end, var_list,
                profiles = ['surf','therm','mid','bot'],
                power_inputs = ['0.1TW', '0.2TW', '0.5TW'],
                power_var_suff = ['0p1TW', '0p2TW', '0p5TW'],
                verbose=False):

    """
    Returns variables containing the ensemble-mean raw data and variables containing the ensemble-mean anomaly data. Anomalies are
    calculated as the difference relative to the control run during the period corresponding to an ensemble member (i.e. the anomalies
    for ensemble member 2 for year 201 to 400 are taking as the difference relative to year 201 to 400 of the control run).

        Args:
            co2_scen (str): one of ['const','doub','quad','const+doub','all']; difference datasets will only be created for the co2 scenario specified, 
                            but ensembles + means may be created for control case of other CO2 scenarios
            avg_period (int): number of years for av/ts period
            mem1_start (int): start year of ens. mem. #1
            mem1_end (int): end year of ens. mem. #1
            var_list (str list): list of variables to read (e.g. var_list = ["temp", "N2", "age", "rhopot2", "salt"])
            profiles (str list): list of profiles to get data for
            power_inputs (str list): list of power inputs to get data for
            power_var_suff (str list): list of variable suffixes for each power input
            verbose: if True, print variable names after declaration
            
        Returns:
            has no return variables, but creates xarray datasets by using myVars = globals()
            
    """
    allowed_scen = ['const','doub','quad','const+doub','all']
    
    if co2_scen not in allowed_scen:
        raise ValueError(f"'co2_scen' must be one of {allowed_scen}.")

    num_ens_mem = 3

    start_yrs = [mem1_start,
                 mem1_start+200,
                 mem1_start+400]
    end_yrs = [mem1_end,
               mem1_end+200,
               mem1_end+400]

    const_exp_root = '_1860IC_200yr_'
    mem1_doub_exp_root = '_2xCO2_1860IC_200yr_' # exp root for 2xCO2 exps is different for mem1 compared to mem2 and mem3
    mem2_3_doub_exp_root = '_2xCO2_200yr_'
    quad_exp_root = '_4xCO2_51-201_'

    ##### CONTROL RUNS #####
    
    ## const CO2 control ##
    const_ctrl_exps = ["tune_ctrl_1860IC_200yr",
                       "tune_ctrl_1860IC_201-2001",
                       "tune_ctrl_1860IC_201-2001"]

    const_ctrl_mem_list, const_ctrl = create_const_doub_ens_mean(const_ctrl_exps,start_yrs,end_yrs,avg_period,var_list,verbose=verbose)
    const_ctrl_name = f"const_ctrl_{mem1_start}_{mem1_end}"
    const_ctrl_mem_list_name = f"{const_ctrl_name}_mem_list"
    myVars.__setitem__(const_ctrl_name, const_ctrl)
    myVars.__setitem__(const_ctrl_mem_list_name, const_ctrl_mem_list)
    if verbose:
        print(f'{const_ctrl_name}, {const_ctrl_mem_list_name} done')

    if co2_scen != 'const':
        ## 2xCO2 control ##
        doub_ctrl_exps = ["tune_ctrl_2xCO2_1860IC_200yr",
                          "ens2_ctrl_2xCO2_200yr",
                          "ens3_ctrl_2xCO2_200yr"]
        
        doub_ctrl_mem_list, doub_ctrl = create_const_doub_ens_mean(doub_ctrl_exps,start_yrs,end_yrs,avg_period,var_list,verbose=verbose)
        doub_ctrl_name = f"doub_ctrl_{mem1_start}_{mem1_end}"
        doub_ctrl_mem_list_name = f"{doub_ctrl_name}_mem_list"
        myVars.__setitem__(doub_ctrl_name, doub_ctrl)
        myVars.__setitem__(doub_ctrl_mem_list_name, doub_ctrl_mem_list)
        if verbose:
            print(f'{doub_ctrl_name}, {doub_ctrl_mem_list_name} done')

        if (co2_scen == 'doub' or co2_scen == 'const+doub' or co2_scen == 'all'):
            # differences compared to constant CO2 control #
            calc_ens_diffs(f"doub_ctrl_{mem1_start}_{mem1_end}_diff",
                           const_ctrl_mem_list,doub_ctrl_mem_list,var_list,verbose=verbose)

        if (co2_scen == 'quad' or co2_scen == 'all'):
            ## 4xCO2 control ##
            quad_ctrl_exps = ["tune_ctrl_4xCO2_51-201",
                              "ens2_ctrl_4xCO2_51-201",
                              "ens3_ctrl_4xCO2_51-201"]
        
            quad_ctrl_mem_list, quad_ctrl = create_quad_ens_mean(quad_ctrl_exps,doub_ctrl_mem_list,doub_ctrl,start_yrs,end_yrs,
                                                                 avg_period,var_list,verbose=verbose)
            quad_ctrl_name = f"quad_ctrl_{mem1_start}_{mem1_end}"
            quad_ctrl_mem_list_name = f"{quad_ctrl_name}_mem_list"
            myVars.__setitem__(quad_ctrl_name, quad_ctrl)
            myVars.__setitem__(quad_ctrl_mem_list_name, quad_ctrl_mem_list)
            if verbose:
                print(f'{quad_ctrl_name}, {quad_ctrl_mem_list_name} done')
        
            # differences compared to constant CO2 and 2xCO2 controls #
            calc_ens_diffs(f"quad_ctrl_{mem1_start}_{mem1_end}_diff_const_ctrl",
                           const_ctrl_mem_list,quad_ctrl_mem_list,var_list,verbose=verbose)
            calc_ens_diffs(f"quad_ctrl_{mem1_start}_{mem1_end}_diff_2xctrl",
                           doub_ctrl_mem_list,quad_ctrl_mem_list,var_list,verbose=verbose)

    
    ##### PERTURBATION RUNS #####
    
    for prof in profiles:
        for index, power_str in enumerate(power_inputs):
            
            # if start_yrs[0] == 1 and end_yrs[0] == 25:
            #     if prof == "therm" and power_str == "0.5TW":
            #         continue #mem3 pp missing
            #     elif prof == "mid":
            #         continue #mem3 pp missing
            #     elif prof == "bot":
            #         continue
                    
            const_exp_name_list = [None] * num_ens_mem
            doub_exp_name_list = [None] * num_ens_mem
            quad_exp_name_list = [None] * num_ens_mem

            const_exp_name_list[0] = prof+const_exp_root+power_str
            const_exp_name_list[1] = "mem2_"+prof+const_exp_root+power_str
            const_exp_name_list[2] = "mem3_"+prof+const_exp_root+power_str
            
            doub_exp_name_list[0] = prof+mem1_doub_exp_root+power_str
            doub_exp_name_list[1] = "ens2_"+prof+mem2_3_doub_exp_root+power_str
            doub_exp_name_list[2] = "ens3_"+prof+mem2_3_doub_exp_root+power_str

            quad_exp_name_list[0] = prof+quad_exp_root+power_str
            quad_exp_name_list[1] = "ens2_"+prof+quad_exp_root+power_str
            quad_exp_name_list[2] = "ens3_"+prof+quad_exp_root+power_str

            const_ens_mem_list, const_ens_mean = create_const_doub_ens_mean(const_exp_name_list,start_yrs,end_yrs,
                                             avg_period,var_list,verbose=verbose)
            if co2_scen != 'const':
                doub_ens_mem_list, doub_ens_mean = create_const_doub_ens_mean(doub_exp_name_list,start_yrs,end_yrs,
                                                                              avg_period,var_list,verbose=verbose)
                if (co2_scen == 'quad' or co2_scen == 'all'):
                    quad_ens_mem_list, quad_ens_mean = create_quad_ens_mean(quad_exp_name_list,doub_ens_mem_list,
                                                                            doub_ens_mean,start_yrs,end_yrs,
                                                                            avg_period,var_list,verbose=verbose)

            ## COMPUTE DIFFERENCES ##
            diff_root = f"{prof}_{power_var_suff[index]}_{mem1_start}_{mem1_end}_diff"

            # differences wrt 1860 control
            const_diff_name = f"const_{diff_root}"
            doub_const_ctrl_diff_name = f"doub_{diff_root}_const_ctrl"
            quad_const_ctrl_diff_name = f"quad_{diff_root}_const_ctrl"

            # differences wrt 1860 experiment with same diffusivity history
            doub_1860_diff_name = f"doub_{diff_root}_1860"
            quad_1860_diff_name = f"quad_{diff_root}_1860"

            # differences wrt control for particular CO2 scenario
            doub_2xctrl_diff_name = f"doub_{diff_root}_2xctrl"
            quad_4xctrl_diff_name = f"quad_{diff_root}_4xctrl"

            ## CONST EXPERIMENTS
            if (co2_scen == 'const' or co2_scen == 'const+doub' or co2_scen == 'all'):
                calc_ens_diffs(const_diff_name,const_ctrl_mem_list,
                               const_ens_mem_list,var_list,verbose=verbose)

            ## 2xCO2 EXPERIMENTS
            if (co2_scen == 'doub' or co2_scen == 'const+doub' or co2_scen == 'all'):
                # differences wrt 1860 control
                calc_ens_diffs(doub_const_ctrl_diff_name,const_ctrl_mem_list,
                               doub_ens_mem_list,var_list,verbose=verbose)
                # differences wrt 1860 experiment with same Kd history
                calc_ens_diffs(doub_1860_diff_name,const_ens_mem_list,
                               doub_ens_mem_list,var_list,verbose=verbose)
                # differences wrt control for particular CO2 scenario
                calc_ens_diffs(doub_2xctrl_diff_name,doub_ctrl_mem_list,
                               doub_ens_mem_list,var_list,verbose=verbose)
                
            ## 4xCO2 EXPERIMENTS
            if (co2_scen == 'quad' or co2_scen == 'all'):
                # differences wrt 1860 control
                calc_ens_diffs(quad_const_ctrl_diff_name,const_ctrl_mem_list,
                               quad_ens_mem_list,var_list,verbose=verbose)
                # differences wrt 1860 experiment with same Kd history
                calc_ens_diffs(quad_1860_diff_name,const_ens_mem_list,
                               quad_ens_mem_list,var_list,verbose=verbose)
                # differences wrt control for particular CO2 scenario
                calc_ens_diffs(quad_4xctrl_diff_name,quad_ctrl_mem_list,
                               quad_ens_mem_list,var_list,verbose=verbose)

                # additional difference calcs for 4xCO2 cases #
                # difference wrt 2xCO2 ctrl
                quad_2xctrl_diff_name = f"quad_{diff_root}_2xctrl"
                calc_ens_diffs(quad_2xctrl_diff_name,doub_ctrl_mem_list,
                               quad_ens_mem_list,var_list,verbose=verbose)
                # difference wrt 2xCO2 experiment with same diffusivity history
                quad_2xCO2_diff_name = f"quad_{diff_root}_2xCO2"
                calc_ens_diffs(quad_2xCO2_diff_name,doub_ens_mem_list,
                               quad_ens_mem_list,var_list,verbose=verbose)
